# Reconnaître les entités nommées
## Entraîner le modèle Bi-LSTM-CRF

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pjox/tutoriel-ner-philologie-computationnelle/blob/master/train_ner_lstm.ipynb)

In [ ]:
# Nous commençons par importer les bibliothèques nécessaires et par télécharger les fichiers d'entraînement et de test.
! pip install flair gdown
! gdown https://drive.google.com/uc\?id\=1NgRHG94lQNZ37TSWJZQHScMc69U8QJ6Z
! unzip mini_presto.zip

Downloading...
From: https://drive.google.com/uc?id=1BpPdP1xDh0ai4Jz71nc8f0IxjboVHokH
To: /Users/portizsu/Code/github.com/pjox/tutoriel-ner-philologie-computationnelle/mini_presto.zip
100%|████████████████████████████████████████| 166k/166k [00:00<00:00, 22.6MB/s]
Archive:  mini_presto.zip
   creating: mini_presto/
  inflating: mini_presto/dev.conll   
  inflating: mini_presto/test.conll  
  inflating: mini_presto/train.conll  


In [1]:
# 1. Nous allons maintenant utiliser la bibliothèque flair pour lire les fichiers d'entraînement et de test.
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# definir les colonnes
columns = {0: 'text', 3: 'ner'}

# c'est le dossier dans lequel sont les fichiers train, test et dev
data_folder = 'mini_presto/'

# initier un corpus en utilisant le format de colonne, le dossier de données et les noms des fichiers train, dev et test
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.conll',
                              test_file='test.conll',
                              dev_file='dev.conll')

2021-12-01 21:25:02,795 Reading data from ftb
2021-12-01 21:25:02,796 Train: ftb/ftb6_train.conll
2021-12-01 21:25:02,796 Dev: ftb/ftb6_dev.conll
2021-12-01 21:25:02,797 Test: ftb/ftb6_test.conll


In [2]:
len(corpus.train)

9881

In [3]:
print(corpus.train[0].to_tagged_string('ner'))

Certes , rien ne dit qu' une seconde motion de censure sur son projet de loi , reprenant l' accord du 10 avril , n' aurait pas été la bonne mais cette probabilité , reconnaissent les socialistes , n' était pas la plus plausible .


In [4]:
from flair.embeddings import WordEmbeddings, FastTextEmbeddings, CharacterEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

# 2. quelle étiquette voulons-nous prédire ?
label_type = 'ner'

# 3. créer le dictionnaire des étiquettes à partir du corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

# 4. initialiser la pile d'embeddings avec FastText et des représentations de caractères
embedding_types = [
    # classical Fasttext embeddings
    WordEmbeddings('fr'),
    # character-level features
    CharacterEmbeddings(),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialiser le tagger de séquences
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

# 6. initialiser l'entraîneur
trainer = ModelTrainer(tagger, corpus)

# 7. Commencer l'entraînement
trainer.train('resources/taggers/ner-lstm-mini-presto',
              learning_rate=0.1,
              mini_batch_size=64,
              max_epochs=5)

2021-12-01 20:44:18,795 Computing label dictionary. Progress:


100%|██████████| 4358/4358 [00:00<00:00, 18370.56it/s]

2021-12-01 20:44:19,078 Corpus contains the labels: ner (#166221)
2021-12-01 20:44:19,079 Created (for label 'ner') Dictionary with 12 tags: O, B-time, I-time, B-org, I-org, B-loc, I-loc, B-pers, I-pers, B-prod, I-prod, B-comp


Dictionary with 12 tags: O, B-time, I-time, B-org, I-org, B-loc, I-loc, B-pers, I-pers, B-prod, I-prod, B-comp
2021-12-01 20:44:33,010 ----------------------------------------------------------------------------------------------------
2021-12-01 20:44:33,011 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('fr')
    (list_embedding_1): CharacterEmbeddings(
      (char_embedding): Embedding(275, 25)
      (char_rnn): LSTM(25, 25, bidirectional=True)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=350, out_features=350, bias=True)
  (rnn): LSTM(350, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=14, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2021-12-01 20:44:33,012 ----------------------------------------------------------------------------------------------------
2021-12-01 20:44:33,012 Corp

{'test_score': 0.0,
 'dev_score_history': [0.34449374747882205,
  0.40752602081665334,
  0.5587734241908007,
  0.6097087378640778,
  0.6494417862838916,
  0.6812121212121212,
  0.6884841000308738,
  0.6941071957358602],
 'train_loss_history': [0.4105785700807559,
  0.201282371233666,
  0.15957253540685326,
  0.13914452762670315,
  0.13207561516787597,
  0.11948045687674902,
  0.10930764609446897,
  0.10562151351311823],
 'dev_loss_history': [tensor(0.2058, device='cuda:0'),
  tensor(0.1530, device='cuda:0'),
  tensor(0.1121, device='cuda:0'),
  tensor(0.1088, device='cuda:0'),
  tensor(0.0921, device='cuda:0'),
  tensor(0.0854, device='cuda:0'),
  tensor(0.0798, device='cuda:0'),
  tensor(0.0733, device='cuda:0')]}

In [ ]:
from flair.data import Sentence

# Charger le modèle fine-tuné
tagger = SequenceTagger.load('resources/taggers/ner-lstm-mini-presto/best-model.pt')

sentence = Sentence("Il étoit gouverneur de Charlemont, qui par la paix deviendra une place très-considérable ; outre cela, il commandoit à Metz.")

# prédire les balises NER
tagger.predict(sentence)

# imprimer une phrase avec des balises prédites
print(sentence.to_tagged_string())